# Object References, Mutability, and Recycling

### Variables Are Not Boxes

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

In [2]:
class Gizmo:
    def __init__(self):
        print(f"Gizmo id: {id(self)}")

In [3]:
x = Gizmo()

Gizmo id: 140121167398480


In [6]:
try:
    y = Gizmo() * 10
except Exception as e:
    print(f"{e=}")

Gizmo id: 140121161008784
e=TypeError("unsupported operand type(s) for *: 'Gizmo' and 'int'")


In [7]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '_1',
 '_5',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__session__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'open',
 'quit',
 'x']

## Identity, Equality and Aliases

In [8]:
charles = {'name':'Charles L. Dodgson', 'born':1832}
lewis = charles
lewis is charles

True

In [9]:
id(charles), id(lewis)

(140121166628864, 140121166628864)

In [11]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [13]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
alex == charles

True

In [14]:
alex is not charles

True

### Choosing Between `==` and `is`

The `is` operator is faster than `==`, because it cannot be overloaded, so Python does not have to find and invoke special methods to evaluate it, and computing is as simple as comparing two integer IDs. In contrast, `a == b` is syntactic sugar for `a.__eq__(b)`.

### The Relative Immutability of Tuples

In [18]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t1 == t2

True

In [19]:
id(t1[-1])

140121236117824

In [20]:
t1[-1].append(99)
t1

(1, 2, [30, 40, 99])

In [21]:
id(t1[-1])

140121236117824

In [22]:
t1 == t2

False

The distinction between equality and identity has further implications when you need to copy an object. A copy is an equal object with a different ID. But if an object contains other objects, should the copy also duplicate the inner objects, or is it OK to share them? There’s no single answer. Read on for a discussion.

## Copies Are Shallow By Default

In [29]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

In [30]:
l2 == l1

True

In [31]:
l2 is l1

False

For lists and other mutable sequences, the shortcut `l2 = l1[:]` also makes a copy.

In [32]:
l1 = [ 3, [66, 55, 44], (7, 8, 9) ]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)

print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]


In [33]:
l2[1] += [33, 22]
l2[2] += (10, 11)

print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


### Deep and Shallow Copies of Arbitrary Objects

Working with shallow copies is not always a problem, but sometimes you need to make deep copies (i.e., duplicates that do not share references of embedded objects). The copy module provides the deepcopy and copy functions that return deep and shallow copies of arbitrary objects.

In [34]:
class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [35]:
import copy
bus1 = Bus( ['Alice', 'Bill', 'Claire', 'David'] )
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(140121161022416, 140121161019088, 140121167379600)

In [36]:
bus1.drop('Bill')
bus2.passengers

['Alice', 'Claire', 'David']

In [38]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(140121166314176, 140121166314176, 140121167294336)

In [39]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

Note that making deep copies is not a simple matter in the general case. Objects may have cyclic references that would cause a naïve algorithm to enter an infinite loop. The `deepcopy` function remembers the objects already copied to handle cyclic refer‐ ences gracefully. 

In [40]:
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [42]:
from copy import deepcopy
c = deepcopy(a)
c

[10, 20, [[...], 30]]

## Function Parameters as References

In [43]:
def f(a, b):
    a += b
    return a

In [44]:
x = 1
y = 2
f(x, y)

3

In [45]:
x, y

(1, 2)

In [46]:
a = [1, 2]
b = [3, 4]
f(a,b)

[1, 2, 3, 4]

In [47]:
a, b

([1, 2, 3, 4], [3, 4])

In [48]:
t = (10, 20)
u = (30, 40)
f(t, u)

(10, 20, 30, 40)

In [49]:
t, u

((10, 20), (30, 40))

### Mutable Types as Parameter Defaults: Bad Idea

In [54]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""
    
    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [55]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [56]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [57]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [58]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [59]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [60]:
bus2.passengers is bus3.passengers

True

In [61]:
bus1.passengers

['Bill', 'Charlie']

In [62]:
dir(HauntedBus.__init__)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [63]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [64]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

### Defensive Programming with Mutable Parameters

In [65]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [66]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

## del and Garbage Collection

In [67]:
import weakref

s1 = {1, 2, 3}
s2 = s1

def bye():
    print('... like tears in the rain.')
    
ender = weakref.finalize(s1, bye)
ender.alive

True

In [68]:
del s1
ender.alive

True

In [69]:
s2 = 'spam'

... like tears in the rain.


In [70]:
ender.alive

False

## Tricks Python Plays with Immutables

I was surprised to learn that, for a tuple `t`, `t[:]` does not make a copy, but returns a reference to the same object. You also get a reference to the same tuple if you write `tuple(t)`.

In [76]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t1

True

In [77]:
t3 = t1[:]
t3 is t1

True

In [78]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1

False

In [79]:
s1 = 'ABC'
s2 = 'ABC'
s2 is s1

True